In [25]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
path = r'C:\Users\fajar\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK066069,Karma Resorts,Other,Past Guests,Karma Kandara,PF296617,Mrs,Jennifer,Soares,Female,...,2024-12-31 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO
1,KK065394,Karma Resorts,Other,Past Guests,Karma Kandara,PF295471,Mr,Christopher,Littlewood,Male,...,2024-12-31 00:00:00,7,HOUSE_USE,V_OWNER_NRB,COMP,5,0,0,Email and Phone,NO
2,KK048313,Karma Resorts,Other,Past Guests,Karma Kandara,PF259658,Mr,Christopher,James,Male,...,2025-01-02 00:00:00,5,HOUSE_USE,V_OWNER,COMP,5,3,0,Email and Phone,NO
3,KK065236,Karma Resorts,Other,Past Guests,Karma Kandara,PF295248,Ms,Susanne,Gerhardsson,Unknown,...,2025-01-04 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
4,KK065235,Karma Resorts,Other,Past Guests,Karma Kandara,PF295247,Mr,Shekhar,Das,Male,...,2025-01-04 00:00:00,5,OTA,OTA,RETAIL,2,1,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,KK000466,Karma Resorts,Other,Past Guests,Karma Kandara,<NA>,<NA>,Holding,Deposits - Events,Unknown,...,2025-01-02 00:00:00,1,COMP,STAFF,COMP,2,0,0,Email Only,NO
130,KK000467,Karma Resorts,Other,Past Guests,Karma Kandara,<NA>,<NA>,Holding,Deposits - Fb,Unknown,...,2025-01-02 00:00:00,1,COMP,STAFF,COMP,2,0,0,Email Only,NO
131,KB046591,Karma Resorts,Other,Past Guests,Karma Bavaria,<NA>,Firma,Karma,Bavaria,Unknown,...,2024-12-31 23:55:00,0,COMP,LGR,GROUP,1,0,0,Email Only,NO
132,KK000468,Karma Resorts,Other,Past Guests,Karma Kandara,<NA>,<NA>,Holding,Deposit - General,Unknown,...,2025-01-02 00:00:00,1,COMP,HOUSE,COMP,2,0,0,Email Only,NO


In [27]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [28]:
df_analytic.fillna('empty', inplace=True)

In [29]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [30]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:90: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [31]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
9,KK058518,Karma Resorts,Other,Past Guests,Karma Kandara,PF280744,Ms,Bianca,Raslan,Female,...,2024-12-30 00:00:00,4,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
17,KK065617,Karma Resorts,Other,Past Guests,Karma Kandara,PF294694,Mr,Jinjin,Jiang,Male,...,2024-12-29 00:00:00,8,OTA,OTA,RETAIL,2,2,0,Email and Phone,NO
34,KB041577,Karma Resorts,Other,Past Guests,Karma Bavaria,PF253144,Herr,Markus,Ferstl,Male,...,2025-01-05 00:00:00,3,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO
35,KB041531,Karma Resorts,Other,Past Guests,Karma Bavaria,PF253144,Herr,Markus,Ferstl,Male,...,2024-12-30 00:00:00,3,OTA,OTA,RETAIL,2,1,0,Email and Phone,NO
43,KB046290,Karma Resorts,Other,Past Guests,Karma Bavaria,PF295888,Frau,Susanne,Welcker,Unknown,...,2025-01-02 00:00:00,2,DIR,TEL,RETAIL,5,0,0,Email and Phone,NO
52,KB046151,Karma Resorts,Other,Past Guests,Karma Bavaria,PF295139,Firma,Karma,Bavaria,Unknown,...,2024-12-29 14:30:00,0,LGR,LGR,GROUP,1,0,0,Email Only,NO
54,KK066552,Karma Resorts,Other,Past Guests,Karma Kandara,PF297585,empty,Shiju,Shaji,Unknown,...,2025-01-04 00:00:00,2,COMP,RESERVED,KC,1,0,0,Email Only,NO
55,KK066553,Karma Resorts,Other,Past Guests,Karma Kandara,PF297585,empty,Shiju,Shaji,Unknown,...,2025-01-04 00:00:00,2,COMP,RESERVED,KC,1,0,0,Email Only,NO
56,KK066690,Karma Resorts,Other,Past Guests,Karma Kandara,PF297585,empty,Shiju,Shaji,Unknown,...,2025-01-05 00:00:00,2,COMP,RESERVED,KC,1,0,0,Email Only,NO
57,KK066684,Karma Resorts,Other,Past Guests,Karma Kandara,PF297585,empty,Shiju,Shaji,Unknown,...,2025-01-05 00:00:00,2,COMP,RESERVED,KC,22,0,0,Email Only,NO


In [32]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK066069,Karma Resorts,Other,Past Guests,Karma Kandara,PF296617,Mrs,Jennifer,Soares,Female,...,2024-12-31 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO
1,KK065394,Karma Resorts,Other,Past Guests,Karma Kandara,PF295471,Mr,Christopher,Littlewood,Male,...,2024-12-31 00:00:00,7,HOUSE_USE,V_OWNER_NRB,COMP,5,0,0,Email and Phone,NO
2,KK048313,Karma Resorts,Other,Past Guests,Karma Kandara,PF259658,Mr,Christopher,James,Male,...,2025-01-02 00:00:00,5,HOUSE_USE,V_OWNER,COMP,5,3,0,Email and Phone,NO
3,KK065236,Karma Resorts,Other,Past Guests,Karma Kandara,PF295248,Ms,Susanne,Gerhardsson,Unknown,...,2025-01-04 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
4,KK065235,Karma Resorts,Other,Past Guests,Karma Kandara,PF295247,Mr,Shekhar,Das,Male,...,2025-01-04 00:00:00,5,OTA,OTA,RETAIL,2,1,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,KK064699,Karma Resorts,Other,Past Guests,Karma Kandara,PF294149,Mr,Phillip,Urbanek,Male,...,2025-01-02 00:00:00,6,HOUSE_USE,V_OWNER_NRB,COMP,4,4,0,Email and Phone,NO
97,KK065237,Karma Resorts,Other,Past Guests,Karma Kandara,PF295249,Mr,Bardia,Danesh,Male,...,2025-01-02 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
98,KK066203,Karma Resorts,Other,Past Guests,Karma Kandara,PF296938,Mr,Um,Jae,Male,...,2025-01-05 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
99,KK066067,Karma Resorts,Other,Past Guests,Karma Kandara,PF296615,empty,Dina,Heryanti,Unknown,...,2024-12-29 00:00:00,3,OTA,OTA,RETAIL,1,0,0,Email and Phone,NO


In [33]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [34]:
find_member_booking.to_csv(r'C:\Users\fajar\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [35]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
19,KB042940,Karma Resorts,Other,Past Guests,Karma Bavaria,PF228618,Frau,Elke,Maier,Female,...,2025-01-02 00:00:00,4,COMP,GER_OWE,COMP,1,0,0,Email and Phone,NO
20,KB042779,Karma Resorts,Other,Past Guests,Karma Bavaria,PF168470,Herr,Joachim,Schug,Male,...,2024-12-31 00:00:00,5,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
21,KB046759,Karma Resorts,Other,Past Guests,Karma Bavaria,PF298112,empty,Alexandra,Babic,Unknown,...,2025-01-02 00:00:00,1,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
22,KB043349,Karma Resorts,Other,Past Guests,Karma Bavaria,PF133894,Herr,Anton,Schwenzer,Male,...,2025-01-01 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email Only,NO
47,KK066542,Karma Resorts,Other,Past Guests,Karma Kandara,PF297585,empty,Shiju,Shaji,Unknown,...,2025-01-04 00:00:00,2,COMP,RESERVED,KC,1,0,0,Email Only,NO
54,KK065220,Karma Resorts,Other,Past Guests,Karma Kandara,PF215766,Mrs,Sharon,Chandra,Female,...,2025-01-03 00:00:00,2,HOUSE_USE,V_OWNER,COMP,6,0,0,Email and Phone,NO


In [36]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [37]:
count_data_akhir = df_analytic.shape[0]

In [38]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK066759,Karma Resorts,Other,Past Guests,Karma Kandara,PF297860,Mr,Colonel Pilot Trinanda,Hasan,Male,...,2024-12-29 00:00:00,1,COMP,GSTAFF,COMP,1,0,0,Email and Phone,NO
1,KB046691,Karma Resorts,Other,Past Guests,Karma Bavaria,PF297950,Frau,Julia,Schmid,Female,...,2024-12-29 00:00:00,1,OTA,OTA,RETAIL,2,1,0,Email and Phone,NO
2,KB046680,Karma Resorts,Other,Past Guests,Karma Bavaria,PF297832,Frau,Sabine,Röske,Female,...,2024-12-29 00:00:00,1,OTA,OTA,RETAIL,1,2,0,Email and Phone,NO
3,KB043365,Karma Resorts,Other,Past Guests,Karma Bavaria,PF279846,Frau,Stefanie,Karnstedt,Female,...,2024-12-29 00:00:00,4,OTA,OTA,RETAIL,2,2,0,Email and Phone,NO
4,KB046544,Karma Resorts,Other,Past Guests,Karma Bavaria,PF297201,Frau,Myrtha,Gesenberg,Female,...,2024-12-29 00:00:00,1,COMP,GER_OWE,COMP,1,0,0,Email Only,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,KSH020132,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF286959,Mrs,Catherine,Nicholls,Female,...,2025-01-05 00:00:00,2,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
91,KSH021799,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF296849,Ms,Vanessa,Cox,Female,...,2025-01-05 00:00:00,3,DIR,GHPA,DIGITAL,2,0,0,Email and Phone,NO
92,KSH021108,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF293176,Mr,Raghavender Reddy,Mutpuri,Male,...,2025-01-05 00:00:00,7,DIR,GHPA,DIGITAL,2,0,0,Email and Phone,NO
93,KSH020698,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF290661,empty,Richard,Long,Male,...,2025-01-05 00:00:00,3,DIR,GHPA,DIGITAL,2,0,0,Email and Phone,NO


In [39]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [40]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar\AppData\Local\Temp\ipykernel_16288\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [41]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK066759,Karma Resorts,Other,Past Guests,Karma Kandara,PF297860,Mr,Colonel Pilot Trinanda,Hasan,Male,...,2024-12-29,1,COMP,GSTAFF,COMP,1,0,0,Email and Phone,NO
1,KSH019860,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF285066,Mrs,Alex And,Jack,Female,...,2024-12-29,1,COMP,WEDDING,GROUP,2,0,0,Email Only,NO
2,KK066819,Karma Resorts,Other,Past Guests,Karma Kandara,PF297946,Mr,Levchyk,Tapac,Male,...,2024-12-29,2,HOUSE_USE,V_OWNER_NRB,COMP,3,0,0,Email and Phone,NO
3,KK066678,Karma Resorts,Other,Past Guests,Karma Kandara,PF297755,Mr,John Gianni,Kovacevic,Male,...,2024-12-29,1,OTA,OTA,RETAIL,1,0,0,Email and Phone,NO
4,KK066845,Karma Resorts,Other,Past Guests,Karma Kandara,PF297759,Mr,Erik,Van Calveen,Male,...,2024-12-29,1,DIR,CHAT,DIGITAL,2,2,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,KK061626,Karma Resorts,Other,Past Guests,Karma Kandara,PF287862,Mr,Brendan,Sarroff,Male,...,2025-01-05,14,DIGITAL,GHPA,DIGITAL,3,2,0,Email and Phone,NO
91,KSH020132,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF286959,Mrs,Catherine,Nicholls,Female,...,2025-01-05,2,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
92,KSH021799,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF296849,Ms,Vanessa,Cox,Female,...,2025-01-05,3,DIR,GHPA,DIGITAL,2,0,0,Email and Phone,NO
93,KSH021108,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF293176,Mr,Raghavender Reddy,Mutpuri,Male,...,2025-01-05,7,DIR,GHPA,DIGITAL,2,0,0,Email and Phone,NO


In [42]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 134
Jumlah Data Yang Duplicate : 33
Jumlah Data Setelah Hapus Duplicate : 101
Jumlah Data Yang Termasuk Member: 6
Jumlah Data Setelah Hapus Member : 95
Jumlah Data Akhir : 95


In [43]:
df_analytic.to_excel(r'C:\Users\fajar\Documents\Python\Data\final_analytics.xlsx', index=False)